In [5]:
# ---------------------------------------------------------
# EDA 
# ---------------------------------------------------------
# Imports 
import json  
import zipfile  

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split



%matplotlib inline 


# That Data
df = pd.read_json('../data.json')
# df.head()

# Make fraud column 
df['fraud'] = df.acct_type.apply(lambda x: 1 if 'fraud' in x else 0)

# Get columns 
columns = list(df.columns)

# See What they looks like 
# df.info()

# Get missing varaiabels 
missing_values = []
for col in columns:
    if df[col].isna().any():
        missing_values.append(col)
        
print(f'Number: {len(missing_values)} \n\nVariables with missing values:\n{missing_values}')

# get variabels by type 
var_floats = []
var_ints = []
var_objects = []

for col in columns:
    if df[col].dtype == 'int64':
        var_ints.append(col)
    elif df[col].dtype == 'float64':
        var_floats.append(col)
    else:
        var_objects.append(col)

print(f' Floats: {var_floats} \n\n Ints: {var_ints} \n\n Objects: {var_objects}')

Number: 12 

Variables with missing values:
['country', 'delivery_method', 'event_published', 'has_header', 'org_facebook', 'org_twitter', 'sale_duration', 'venue_country', 'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state']
 Floats: ['delivery_method', 'event_published', 'gts', 'has_header', 'org_facebook', 'org_twitter', 'sale_duration', 'venue_latitude', 'venue_longitude'] 

 Ints: ['approx_payout_date', 'body_length', 'channels', 'event_created', 'event_end', 'event_start', 'fb_published', 'has_analytics', 'has_logo', 'name_length', 'num_order', 'num_payouts', 'object_id', 'sale_duration2', 'show_map', 'user_age', 'user_created', 'user_type', 'fraud'] 

 Objects: ['acct_type', 'country', 'currency', 'description', 'email_domain', 'listed', 'name', 'org_desc', 'org_name', 'payee_name', 'payout_type', 'previous_payouts', 'ticket_types', 'venue_address', 'venue_country', 'venue_name', 'venue_state']


In [3]:
columns = list(df.columns)
columns

['acct_type',
 'approx_payout_date',
 'body_length',
 'channels',
 'country',
 'currency',
 'delivery_method',
 'description',
 'email_domain',
 'event_created',
 'event_end',
 'event_published',
 'event_start',
 'fb_published',
 'gts',
 'has_analytics',
 'has_header',
 'has_logo',
 'listed',
 'name',
 'name_length',
 'num_order',
 'num_payouts',
 'object_id',
 'org_desc',
 'org_facebook',
 'org_name',
 'org_twitter',
 'payee_name',
 'payout_type',
 'previous_payouts',
 'sale_duration',
 'sale_duration2',
 'show_map',
 'ticket_types',
 'user_age',
 'user_created',
 'user_type',
 'venue_address',
 'venue_country',
 'venue_latitude',
 'venue_longitude',
 'venue_name',
 'venue_state',
 'fraud']

In [25]:
# previous payouts really good indication of whether something is fraud
# almost all that are not fraud have previous payouts
df['user_age']

0          36
1         149
2         214
3         889
4          35
         ... 
14332       0
14333    1374
14334      84
14335     812
14336       0
Name: user_age, Length: 14337, dtype: int64

predicted,0,1
actual,,
0,3251,19
1,73,242


In [8]:
df['previous_payouts']

0                                                       []
1        [{'name': 'RUF', 'created': '2010-10-01 01:10:...
2        [{'name': 'Danielle Severn', 'created': '2010-...
3        [{'name': '', 'created': '2010-11-09 01:10:15'...
4        [{'name': 'Arts and Business Council or Greate...
                               ...                        
14332                                                   []
14333    [{'name': 'ARCS Foundation Inc., San Diego Cha...
14334    [{'name': '', 'created': '2013-05-16 03:26:11'...
14335    [{'name': '', 'created': '2010-11-30 01:11:30'...
14336                                                   []
Name: previous_payouts, Length: 14337, dtype: object

In [29]:
# X is features, y is target. Returns a cross tab of predicted 
# and actual values for fraud vs not fraud
def get_crosstab(X, y, model_type):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
    model = model_type
    model.fit(Xtrain, ytrain)
    predicted = model.predict(Xtest)
    actual = ytest
    return pd.crosstab(ytest, model.predict(Xtest), rownames=['actual'], colnames=['predicted'])

In [30]:
from sklearn.linear_model import LogisticRegression

mask = df.previous_payouts.apply(lambda x: 0 if len(x) <= 0 else 1)
df['mask_previous_payout'] = mask
X = df[['mask_previous_payout', 'user_age']]
y = df['fraud']

# get predict proba from both models, for each data point take average 
# probabilities. probs set at .5 by default

get_crosstab(X, y, LogisticRegression())

predicted,0,1
actual,,
0,3225,25
1,77,258


In [31]:
from sklearn.tree import DecisionTreeClassifier
get_crosstab(X, y, DecisionTreeClassifier())

predicted,0,1
actual,,
0,3240,18
1,73,254


In [ ]:
# ROC curve 